**Load Dependencies**

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, Dropout, Dense
from tensorflow.keras.layers import Flatten 

**Construct CNN** 

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(4, 4),
                activation='relu', #kernel_regularizer=regularizers.l2(0.04),
                input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 4)))
model.add(Conv2D(64, (3, 5), activation='relu'
               , kernel_regularizer=regularizers.l2(0.04)
               ))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (2, 2), activation='relu'
  # , kernel_regularizer=regularizers.l2(0.04)
   ))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.04)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.04)))
model.add(Dense(num_genres, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
             metrics=[metric])del = Sequential()
model.add(Conv2D(64, kernel_size=(4, 4),
             activation='relu', #kernel_regularizer=regularizers.l2(0.04),
             input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 4)))
model.add(Conv2D(64, (3, 5), activation='relu'
            , kernel_regularizer=regularizers.l2(0.04)
            ))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (2, 2), activation='relu'
# , kernel_regularizer=regularizers.l2(0.04)
))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.04)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.04)))
model.add(Dense(num_genres, activation='softmax'))

Compile Model

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
          optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
          metrics=[metric])
